1) Посчитайте, сколько компаний закрылось.

In [ ]:
select count(*)
from company
where status = 'closed'

2) Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
select funding_total
from company
where category_code = 'news' and country_code = 'USA'
order by funding_total desc

3) Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
select sum (price_amount)
from acquisition
where term_code = 'cash' and
extract(year from cast(acquired_at as timestamp)) in (2011, 2012, 2013)

4) Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
select first_name,
        last_name,
        twitter_username
from people
where twitter_username like 'Silver%'

5) Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
select *
from people
where twitter_username like '%money%' 
and last_name like 'K%'

6) Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
select country_code,
        sum(funding_total) as sf
from company
group by country_code
order by sf desc

7) Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.

Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
select funded_at,
        min(raised_amount),
        max(raised_amount)
from funding_round
group by funded_at
having min(raised_amount) <> 0 
and min(raised_amount) <>  max(raised_amount)

8) Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
select *,
        case
            when invested_companies >= 100 then 'high_activity'
            when invested_companies >= 20 and invested_companies < 100 then 'middle_activity'
            when invested_companies < 20 then 'low_activity'
        end
from fund

9) Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
select activ.activity,
        round(avg(activ.investment_rounds)) as round
from 
(SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund) as activ
group by activ.activity
order by round

10) Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
select country_code,
        min(invested_companies),
        max(invested_companies),
        avg(invested_companies)
from fund
where extract(year from cast(founded_at as timestamp)) between 2010 and 2012
group by country_code
having min(invested_companies) <> 0
order by avg(invested_companies) desc
limit 10

11) Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
select p.first_name,
        p.last_name,
        e.instituition
from people as p
left join education as e on e.person_id = p.id

12) Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
select c.name,
        count(distinct e.instituition)
from company as c
inner join people as p on p.company_id = c.id
inner join education as e on e.person_id = p.id
group by c.name
order by count(distinct e.instituition) desc
limit 5

13) Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
select distinct c.name
from company as c
left join funding_round as fd on c.id = fd.company_id
where c.status = 'closed'
and fd.is_first_round = 1
and fd.is_last_round = 1

14) Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
select distinct p.id
from people as p
where p.company_id in
(select distinct c.id
from company as c
left join funding_round as fd on c.id = fd.company_id
where c.status = 'closed'
and fd.is_first_round = 1
and fd.is_last_round = 1)

15) Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
select distinct p.id,
        e.instituition
from people as p
join education as e on e.person_id = p.id
where p.company_id in
(select distinct c.id
from company as c
left join funding_round as fd on c.id = fd.company_id
where c.status = 'closed'
and fd.is_first_round = 1
and fd.is_last_round = 1)

16) Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
select p.id,
        count(e.instituition)
from people as p
join education as e on e.person_id = p.id
where p.company_id in
(select distinct c.id
from company as c
left join funding_round as fd on c.id = fd.company_id
where c.status = 'closed'
and fd.is_first_round = 1
and fd.is_last_round = 1)
group by p.id

17) Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
select avg(t1.number)
from (select p.id,
        count(e.instituition) as number
from people as p
join education as e on e.person_id = p.id
where p.company_id in
(select distinct c.id
from company as c
left join funding_round as fd on c.id = fd.company_id
where c.status = 'closed'
and fd.is_first_round = 1
and fd.is_last_round = 1)
group by p.id) as t1

18) Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook

In [ ]:
SELECT AVG(n.inst)
FROM (SELECT p.id, 
       COUNT(e.instituition) AS inst
FROM people AS p 
JOIN education AS e ON p.id = e.person_id
WHERE company_id in (SELECT id
                     FROM company 
                     WHERE name='Facebook')                   
GROUP BY p.id) As n;

19) Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
select f.name as name_of_fund,
        c.name as name_of_company,
        fr.raised_amount as amount
FROM company AS c
JOIN investment AS i ON c.id = i.company_id
JOIN fund AS f ON i.fund_id = f.id
JOIN funding_round AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6
AND EXTRACT(YEAR FROM fr.funded_at::date) between '2012' and '2013'

20) Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
select
c.name,
a.price_amount,
c_1.name,
c_1.funding_total,
round(a.price_amount/c_1.funding_total)
from acquisition a
LEFT JOIN company AS c ON a.acquiring_company_id = c.id
LEFT JOIN company AS c_1 ON a.acquired_company_id = c_1.id
where a.price_amount>0 and c_1.funding_total>0
order by a.price_amount desc
limit 10

21) Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM CAST(fr.funded_at as date))
FROM company AS c 
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.category_code = 'social'
And EXTRACT(YEAR FROM CAST (fr.funded_at as date)) BETWEEN 2010 AND 2013

22) Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH funds AS (SELECT EXTRACT(MONTH FROM fr.funded_at) AS MONTH,
                       COUNT(DISTINCT f.NAME) AS funds
                FROM funding_round AS fr
                JOIN investment AS i ON fr.id = i.funding_round_id
                JOIN (SELECT id,
                             NAME
                     FROM fund
                     WHERE country_code = 'USA') AS f ON i.fund_id = f.id
                WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
                GROUP BY MONTH),
     coms AS (SELECT EXTRACT(MONTH FROM acquired_at) AS MONTH,
                         COUNT(acquired_at) AS acquired_companies,
                         SUM(price_amount) AS total_amount
                FROM acquisition
                WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
                GROUP BY MONTH)
SELECT funds.MONTH,
       funds.funds,
       coms.acquired_companies,
       coms.total_amount
FROM funds
JOIN coms ON funds.MONTH = coms.MONTH

23) Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
SELECT i_2011.country, 
       year_2011, 
       year_2012, 
       year_2013
FROM
     (SELECT AVG(funding_total) AS year_2011,     
             country_code AS country
      FROM company
      WHERE EXTRACT(YEAR FROM founded_at)= 2011
      GROUP BY country) AS i_2011
JOIN(SELECT AVG(funding_total) AS year_2012,     
                country_code AS country
      FROM company
      WHERE EXTRACT(YEAR FROM founded_at)= 2012
      GROUP BY country) AS i_2012  ON i_2011.country=i_2012.country
JOIN(SELECT AVG(funding_total) AS year_2013,     
                country_code AS country
      FROM company
      WHERE EXTRACT(YEAR FROM founded_at)= 2013
      GROUP BY country) AS i_2013 ON i_2012.country=i_2013.country
ORDER BY year_2011 desc;